<a href="https://colab.research.google.com/github/ShinyaKatoh/Trial_Polarity/blob/main/plot_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ShinyaKatoh/Trial_Polarity
!pip install einops
!pip install torchinfo

In [ ]:
# Google driveにマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/Trial_Polarity')

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F
import torch.nn.init as init

import os
import glob
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

import util

In [ ]:
class AIdetector:
    def __init__(self, model_path):
        # device指定
        self.device = torch.device('cpu')
        # モデルパラメータのロード
        self.model = torch.load(model_path, map_location=self.device, weights_only=False)
        # モデルを評価モードへの着替え
        self.model.eval()

    def pred(self, input_data):
        with torch.no_grad():
            inputs = input_data.float().to(self.device)
            outputs = self.model(inputs)
        return outputs

In [ ]:
waves = torch.load('/content/Trial_Polarity/data/test_data.pt')
labels_CLA = torch.load('/content/Trial_Polarity/data/test_label_for_CLA.pt')
labels_SEG = torch.load('/content/Trial_Polarity/data/test_label_for_SEG.pt')

In [ ]:
detector = AIdetector('/content/drive/MyDrive/model1/model_epoch_100.pth')

In [ ]:
c = 0
for i in range(10):
# for i in range(200):

    wave = waves[i:i+1]
    label_CLA = labels_CLA[i:i+1]
    label_SEG = labels_SEG[i:i+1]


    output = detector.pred(wave)
    # print(output)

    if torch.max(output,1)[1] == 0:
        p_pola = 'U'
    elif torch.max(output,1)[1] == 1:
        p_pola = 'D'
    elif torch.max(output,1)[1] == 2:
        p_pola = 'N'

    if torch.max(label_CLA,1)[1] == 0:
        h_pola = 'U'
    elif torch.max(label_CLA,1)[1] == 1:
        h_pola = 'D'
    elif torch.max(label_CLA,1)[1] == 2:
        h_pola = 'N'

    if p_pola == h_pola:

        c += 1

        fig = plt.figure(figsize=(10,4))
        # plt.subplot(2,1,1)
        plt.plot(wave[0][0,:].numpy(), c='black')
        plt.title('HUMAN : ' + h_pola + '   -    PRED : ' + p_pola+'   PROB : '+ str( round(torch.max(output,1)[0].item()*100,2) ) + ' %')

        if h_pola == 'U' or h_pola == 'D':
            pidx = torch.max(label_SEG[0,:],1)[1]
            print(pidx)
            plt.axvline(x=pidx[0], c='red')

        # plt.subplot(2,1,2)
        # plt.plot(label_SEG[0][0,:].numpy())

        plt.show()
        plt.close()

        if c == 30:
          break

print(c/waves.shape[0])